# Example on Credit Risk data set
Here you can find a simple example on how to setup and run CoGS to find a counterfactual explanation for an application concerning financial credit risk

### Setup libraries

In [1]:
from cogs.evolution import Evolution
from cogs.fitness import gower_fitness_function
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

We also set up the random seed for reproducibility

In [2]:
SEED = 42
np.random.seed(SEED)

### Data set
Next, we load the data set "South German Credit", which concerns learning a model of whether providing a financial credit to a user may be risky.

See https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29 for more info.

In [3]:
# Load data set & do some pre-processing
df = pd.read_csv("south_german_credit.csv")
categorical_feature_names = ['purpose', 'personal_status_sex',
    'other_debtors', 'other_installment_plans', 'telephone', 'foreign_worker']
# Note: some other features are indices (categories in which the order matters), treated as numerical here for simplicity
label_name = 'credit_risk'
desired_class = 1 # this means "low risk"

for feat in categorical_feature_names:
    df[feat] = pd.Categorical(df[feat])
    df[feat] = df[feat].cat.codes
feature_names = list(df.columns)
feature_names.remove(label_name)

# Prepare data to be in numpy format, as typically used to train a scikit-learn model
X = df[feature_names].to_numpy()
y = df[label_name].to_numpy().astype(int)
# Assume we have a specific train & test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)

### Train the model
We train a random forest model that acts as the "black-box" that, e.g., a bank may use to (help) decide whether to grant the credir or not.

In [4]:

# Train black-box model (bbm)
# (we do not one-hot-encode here for simplicity, but you can use robust_cfe/blackbox_with_preproc.py to easily do that)
bbm = RandomForestClassifier(random_state=SEED)
bbm.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

### Pick the user
Next, we simulate to have a user for whom the decision of the black-box model is undesired. 
For example, let's pick the last point in the test set for which the prediction is unfavourable.

In [5]:

# Let's consider, e.g., the last test sample for which an undesired decision is given
p = bbm.predict(X_test)
idx = np.argwhere(p != desired_class).squeeze()[-1]
x = X_test[idx] # this is our unhappy user!

### CoGS
We are almost ready to now use CoGS to find a counterfactual explanation for our unhappy user `x`.
First, though, we need to provide CoGS with:
1) Intervals within which the search takes place (for categorical features, which categories are possible)
2) The indices of categorical features
3) Potential plausibility constraints (e.g., the age of a person cannot become lower than it is)

All of these three must be provided as lists that have the same order, in particular the order used to list the feature in `X_train` and `X_test`.

In [6]:

# Some preparation for CoGS

# Set up search bounds
feature_intervals = list()
for i, feat in enumerate(feature_names):
  if feat in categorical_feature_names:
    interval_i = np.unique(X_train[:,i])
  else:
    interval_i = (np.min(X_train[:,i]), np.max(X_train[:,i]))
  feature_intervals.append(interval_i)

# Set up which feature indices are categorical
indices_categorical_features = [i for i, feat in enumerate(feature_names) if feat in categorical_feature_names]

# Let's also set up a plausibility constraint for the feature "age"
pcs = ['>=' if feat=='age' else None for feat in feature_names]

We can now setup the hyper-parameters of CoGS, and then run the search!
We put some comments to explain what they mean in the code below

In [7]:
cogs = Evolution(
        # hyper-parameters of the problem (required!)
        x=x,  # the unhappy user
        fitness_function=gower_fitness_function,  # a classic fitness function for counterfactual explanations
        fitness_function_kwargs={'blackbox':bbm,'desired_class': desired_class},  # these must be passed for the fitness function to work
        feature_intervals=feature_intervals,  # intervals within which the search operates
        indices_categorical_features=indices_categorical_features,  # the indices of the features that are categorical
        plausibility_constraints=pcs, # can be "None" if no constraints need to be set
        # hyper-parameters of the evolution (all optional)
        evolution_type='classic', # the type of evolution, classic works quite  well
        population_size=1000,   # how many candidate counterfactual examples to evolve simultaneously
        n_generations=100,  # number of iterations for the evolution
        selection_name='tournament_4', # selection pressure
        init_temperature=0.8, # how "far" from x we initialize
        num_features_mutation_strength=0.25, # strength of random mutations for numerical features
        num_features_mutation_strength_decay=0.5, # decay for the hyper-param. above
        num_features_mutation_strength_decay_generations=[50,75,90], # when to apply the decay
        # others
        verbose=True  # logs progress at every generation 
)

Ready to run!

In [9]:
cogs.run()

generation: 1 best fitness: -0.293837432487357 avg. fitness: -0.5214562185233141
generation: 2 best fitness: -0.2510893680668098 avg. fitness: -0.45388250049116696
generation: 3 best fitness: -0.2323692193952797 avg. fitness: -0.40303163962654004
generation: 4 best fitness: -0.20012311659357698 avg. fitness: -0.34679752948384396
generation: 5 best fitness: -0.14970165842315736 avg. fitness: -0.29572311736676454
generation: 6 best fitness: -0.14473293186032618 avg. fitness: -0.2542540768340204
generation: 7 best fitness: -0.09679574332982878 avg. fitness: -0.2142091158529426
generation: 8 best fitness: -0.08342069842620103 avg. fitness: -0.1790862971543234
generation: 9 best fitness: -0.05696144238579127 avg. fitness: -0.14657272079943426
generation: 10 best fitness: -0.030340551858586248 avg. fitness: -0.11739392672781657
generation: 11 best fitness: -0.030207758431344642 avg. fitness: -0.09339445268296587
generation: 12 best fitness: -0.02911020994296238 avg. fitness: -0.0751850959768

## Counterfactual explanation
Now that CoGS has terminated, we can look at its result.
The field `cogs.elite` contains the best-found counterfactual example, i.e., a point `x'` for which `bbm(x')=desired_class`.
The relative counterfactual explanation is simply `x'-x` (there exist more involved definitions of counterfactual explanations, here we use this simple one).
Let's take a look at what the user needs to do to obtain the desired class, i.e., be granted the loan.

In [11]:
# Get the best-found counterfactual example (called elite)
cf_example = cogs.elite
cf_explanation = cogs.elite - x

# Show counterfactual explanation
if bbm.predict([cf_example])[0] == desired_class:
  print("Success! Here's the explanation:")
  for i, feat in enumerate(feature_names):
    cf_expl_i = cf_explanation[i]
    if cf_expl_i != 0:
      print(" Feature {} should change by {}{:.5f}".format(feat,"+" if np.sign(cf_expl_i) > 0 else "-",cf_expl_i))
else:
  print("Failed to find a counterfactual explanation for the desired class :(")

Success! Here's the explanation:
 Feature savings should change by +0.50079
